In [79]:
import os
import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import math

from matplotlib import dates
import datetime

from heapq import nsmallest

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import statsmodels.api as sm

import prophet
from prophet.diagnostics import cross_validation
from prophet.diagnostics import performance_metrics
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy.optimize import curve_fit
import seaborn as sns
from neuralprophet import NeuralProphet, set_log_level

import warnings

In [2]:
holidays=['2020-01-01','2020-01-06','2020-04-12','2020-04-13','2020-05-01','2020-05-03',
          '2020-05-31', '2020-06-11','2020-08-15','2020-11-01','2020-11-11','2020-12-25',
          '2020-12-26','2021-01-01','2021-01-06','2021-04-04','2021-04-05','2021-05-01',
          '2021-05-03','2021-05-23','2021-06-03','2021-08-15','2021-11-01','2021-11-11',
          '2021-12-25','2021-12-26', '2022-01-01','2022-01-06', '2022-04-01','2022-04-02','2022-04-03','2022-04-04','2022-04-05',
          '2022-04-27','2022-04-28','2022-04-29','2022-04-30',
          '2022-05-01', '2022-05-02','2022-05-03','2022-05-04',
          '2022-05-23','2022-06-03','2022-08-15','2022-11-01','2022-11-11',
          '2022-12-25','2022-12-26'
         ]

state = pd.DataFrame({'ds': pd.to_datetime(holidays),
                      'holiday': 'state_holiday'
                      })
holidays = state

In [60]:
path = os.getcwd()
files = os.listdir(path)
files_csv = [f for f in files if f[:6] == 'zakupy']

In [4]:
df = pd.DataFrame()
for f in files_csv:
    data = pd.read_excel(f)
    df = df.append(data)

df.head(3)

WARNING - (py.warnings._showwarnmsg) - /tmp/ipykernel_94068/1046609750.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)

WARNING - (py.warnings._showwarnmsg) - /tmp/ipykernel_94068/1046609750.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)

WARNING - (py.warnings._showwarnmsg) - /tmp/ipykernel_94068/1046609750.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(data)



,NumerZlecenia,LokalizacjaAkcji,LokalizacjaPoczatkowa,LokalizacjaKoncowa,CzasPrzejazdu,PrzebytyDystans,SredniaPredkosc,OcenaJazdyExpress,OcenaPredkosciExpress,OcenaOgolnaExpress,...,SamochodId,NumerRejestracyjny,Subsystem,Stan,DataDodania,Uwagi,SklepTrafiszopId,NazwaSklepu,AdresSklepu,MiastoSklepu
0,ZT/2021/2248344,Kraków,"Kraków, Tyniecka 188","Kraków, Tyniecka 188",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,brak,Zamkniete,2021-01-01 12:05:50.7683578,Osoba kontaktowa: 508905857 | Godzina dostar...,4,SPOŁEM Kraków,Daszyńskiego 3 31-537 Kraków,Kraków
1,ZT/2021/2248353,Łódź,"Łódź, ul. Żeligowskiego 8/10 - 24h","Łódź, ul. Żeligowskiego 8/10 - 24h",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,brak,Zamkniete,2021-01-01 12:34:11.5504132,Osoba kontaktowa: 608824286 | Godzina dostar...,28,Carrefour Bandurskiego,Bandurskiego 49 94-020 Łódź,Łódź
2,ZT/2021/2248355,Katowice,"Katowice, Siemianowicka 52C - 24h","Katowice, Siemianowicka 52C - 24h",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,brak,Zamkniete,2021-01-01 13:02:39.4225176,Osoba kontaktowa: 511461537 Notatka klienta: ...,3,SPOŁEM Katowice,Mickiewicza 4 40-954 Katowice,Katowice


In [6]:
df=df.reset_index()
del df['index']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 155773 entries, 0 to 155772
Data columns (total 36 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   NumerZlecenia                 155773 non-null  object 
 1   LokalizacjaAkcji              154854 non-null  object 
 2   LokalizacjaPoczatkowa         155773 non-null  object 
 3   LokalizacjaKoncowa            155773 non-null  object 
 4   CzasPrzejazdu                 872 non-null     float64
 5   PrzebytyDystans               872 non-null     float64
 6   SredniaPredkosc               872 non-null     float64
 7   OcenaJazdyExpress             872 non-null     float64
 8   OcenaPredkosciExpress         872 non-null     float64
 9   OcenaOgolnaExpress            872 non-null     float64
 10  NajwyzszaPredkosc             872 non-null     float64
 11  OcenaRezerwacjiCms            0 non-null       float64
 12  IdWsparciaCms                 0 non-null    

In [7]:
#Lokalizacje akcji zapełnic miastem sklepu
df.LokalizacjaAkcji.fillna(df.MiastoSklepu, inplace=True)
#Wyrzucenie kolumn z brakiem DatyRozpoczęcia / data rozpoczęcia może wystąpić w następnych dniach od planowanej
df = df.dropna(subset=['DataRozpoczecia'])
df=df.reset_index()
del df['index']

In [8]:
# zmiana formatu daty na datetime
df['PlanowanaDataRozpoczecia']= pd.to_datetime(df['PlanowanaDataRozpoczecia'])
df['PlanowanaDataZakonczenia']= pd.to_datetime(df['PlanowanaDataZakonczenia'])
df['DataZakonczenia']= pd.to_datetime(df['DataZakonczenia'])
df['ZadeklarowanaData']= pd.to_datetime(df['ZadeklarowanaData'])
df['DataRozpoczecia']= pd.to_datetime(df['DataRozpoczecia'])
df['DataDodania']= pd.to_datetime(df['DataDodania'])

In [9]:
#Wyrzucenie kolumn gdzie data zakonczenia jest przed data rozpoczecia
df = df[df['DataZakonczenia']>df['DataRozpoczecia']]
print(len(df))

152371


In [12]:
#Dodanie kolumny z numeryczna wartością dla dnia tygodnia(0-poniedziałek itd.)
#df['day_of_week'] = df['PlanowanaDataRozpoczecia'].dt.day_name() #wartość string
df['day_of_week'] = df['PlanowanaDataRozpoczecia'].dt.dayofweek #wartosć numeryczna
df['Lp'] = 1 

In [13]:
# usunięcie niedziel
df_weekdays = df[df['DataRozpoczecia'].dt.dayofweek < 6]
# df_weekdays = df
df_weekdays = df_weekdays.reset_index()
df_weekdays = df_weekdays.drop(columns=['index'])


In [14]:
# Lista miast
city_list = ['Kraków' 'Łódź' 'Katowice' 'Warszawa' 'Wrocław' 'Poznań' 'Gdańsk'
 'Chorzów' 'Białystok' 'Lublin' 'Gliwice' 'Szczecin' 'Sosnowiec'
 'Bielsko-Biała' 'Piotrków Trybunalski' 'Bytom' 'Jaworzno' 'Rybnik'
 'Wołomin' 'Radom' 'Mińsk Mazowiecki']

big_city_list = ['Warszawa', 'Kraków', 'Wrocław']

In [15]:
#Sprzedaż danego dnia tygodnia
data_sales_weekday = df.groupby(df['day_of_week']).agg({"Lp":"sum"})
#Sprzedaż w danych godzinach
data_sales_hour = df.groupby(df['DataRozpoczecia'].dt.hour).agg({"Lp":"sum"})
#Sprzedaż danego dnia miesiąca
data_sales_day = df.groupby(df['DataRozpoczecia'].dt.day).agg({"Lp":"sum"})
#Sprzedaż danego miesiąca
data_sales_month = df.groupby(df['DataRozpoczecia'].dt.month).agg({"Lp":"sum"})
#Sprzedaż danego roku
data_sales_year = df.groupby(df['DataRozpoczecia'].dt.year).agg({"Lp":"sum"})
#Sprzedaż w danych godzinach - Data zakończenia
data_sales_hour_z = df.groupby(df['DataZakonczenia'].dt.hour).agg({"Lp":"sum"})
#Sprzedaż danego tygodnia w roku
data_sales_week = df.groupby(df['DataRozpoczecia'].dt.week).agg({"Lp":"sum"})

WARNING - (py.warnings._showwarnmsg) - /tmp/ipykernel_94068/1539491939.py:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data_sales_week = df.groupby(df['DataRozpoczecia'].dt.week).agg({"Lp":"sum"})



In [16]:
# podzial na miasta
df_warszawa = df.loc[df['MiastoSklepu'] == 'Warszawa']
df_krakow = df.loc[df['MiastoSklepu'] == 'Kraków']
df_wroclaw = df.loc[df['MiastoSklepu'] == 'Wrocław']


In [17]:
#podział na dni

date_sales = df.groupby(pd.to_datetime(df['DataRozpoczecia']).dt.date).agg({"Lp":"sum"})
warsaw_date_sales = df_warszawa.groupby(pd.to_datetime(df_warszawa['DataRozpoczecia']).dt.date).agg({"Lp":"sum"})
krakow_date_sales = df_krakow.groupby(pd.to_datetime(df_krakow['DataRozpoczecia']).dt.date).agg({"Lp":"sum"})
wroclaw_date_sales = df_wroclaw.groupby(pd.to_datetime(df_wroclaw['DataRozpoczecia']).dt.date).agg({"Lp":"sum"})

In [18]:
# odszumianie danych - uzupelnianie dat pustych, lub o wartosci mniejszej niz
# 10% sredniej ilosci z danego miesiaca poprzez interpolacje "niemalymi"
# wartosciami z okolicznych dat

years = [2020, 2021, 2022]
idx = pd.date_range('2020-10-30', '2022-08-25')
wwa_idx = pd.date_range('2020-10-30', '2022-08-25')
krk_idx = pd.date_range('2020-10-30', '2022-08-25')
wr_idx = pd.date_range('2020-10-30', '2022-08-25')


year_sales_dict = {}
year_sales_dict_week = {}

year_sales_dict_wwa = {}
year_sales_dict_week_wwa = {}

year_sales_dict_wr = {}
year_sales_dict_week_wr = {}

year_sales_dict_krk = {}
year_sales_dict_week_krk = {}

date_sales_un = date_sales.reindex(idx, fill_value=0)
date_sales_un = date_sales_un.reset_index()

date_sales_wwa_un = warsaw_date_sales.reindex(idx, fill_value=0)
date_sales_wwa_un = date_sales_wwa_un.reset_index()

date_sales_krk_un = krakow_date_sales.reindex(idx, fill_value=0)
date_sales_krk_un = date_sales_krk_un.reset_index()

date_sales_wr_un = wroclaw_date_sales.reindex(idx, fill_value=0)
date_sales_wr_un = date_sales_wr_un.reset_index()

date_sales_un = date_sales_wwa_un.rename(columns={'index': 'ds', 'Lp': 'y'})
date_sales_wwa_un = date_sales_wwa_un.rename(columns={'index': 'ds', 'Lp': 'y'})
date_sales_krk_un = date_sales_krk_un.rename(columns={'index': 'ds', 'Lp': 'y'})
date_sales_wr_un = date_sales_wr_un.rename(columns={'index': 'ds', 'Lp': 'y'})

if 'DataRozpoczecia' in date_sales_un.columns.values.tolist():
    date_sales_un.drop(['DataRozpoczecia'], axis=1)
    
if 'DataRozpoczecia' in date_sales_wwa_un.columns.values.tolist():
    date_sales_wwa_un.drop(['DataRozpoczecia'], axis=1)
    
if 'DataRozpoczecia' in date_sales_krk_un.columns.values.tolist():
    date_sales_krk_un.drop(['DataRozpoczecia'], axis=1)
    
if 'DataRozpoczecia' in date_sales_wr_un.columns.values.tolist():
    date_sales_wr_un.drop(['DataRozpoczecia'], axis=1)

for year in years:
    year_sales_dict[year]=df[df['DataRozpoczecia'].dt.year == year].groupby(df['DataRozpoczecia'].dt.month).agg({"Lp":"sum"})
    year_sales_dict_week[year]=df[df['DataRozpoczecia'].dt.year == year].groupby(df['DataRozpoczecia'].dt.week).agg({"Lp":"sum"})
    
    year_sales_dict_wwa[year]=df_warszawa[df_warszawa['DataRozpoczecia'].dt.year == year].groupby(df_warszawa['DataRozpoczecia'].dt.month).agg({"Lp":"sum"})
    year_sales_dict_week_wwa[year]=df_warszawa[df_warszawa['DataRozpoczecia'].dt.year == year].groupby(df_warszawa['DataRozpoczecia'].dt.week).agg({"Lp":"sum"})

    year_sales_dict_krk[year]=df_krakow[df_krakow['DataRozpoczecia'].dt.year == year].groupby(df_krakow['DataRozpoczecia'].dt.month).agg({"Lp":"sum"})
    year_sales_dict_week_krk[year]=df_krakow[df_krakow['DataRozpoczecia'].dt.year == year].groupby(df_krakow['DataRozpoczecia'].dt.week).agg({"Lp":"sum"})

    year_sales_dict_wr[year]=df_wroclaw[df_wroclaw['DataRozpoczecia'].dt.year == year].groupby(df_wroclaw['DataRozpoczecia'].dt.month).agg({"Lp":"sum"})
    year_sales_dict_week_wr[year]=df_wroclaw[df_wroclaw['DataRozpoczecia'].dt.year == year].groupby(df_wroclaw['DataRozpoczecia'].dt.week).agg({"Lp":"sum"})

    
for key, value in year_sales_dict.items():
    value = value.to_dict()['Lp']
    year_sales_dict[key] = value
    
for key, value in year_sales_dict_week.items():
    value = value.to_dict()['Lp']
    year_sales_dict_week[key] = value
    
for key, value in year_sales_dict_wwa.items():
    value = value.to_dict()['Lp']
    year_sales_dict_wwa[key] = value
    
for key, value in year_sales_dict_week_wwa.items():
    value = value.to_dict()['Lp']
    year_sales_dict_week_wwa[key] = value
    
for key, value in year_sales_dict_krk.items():
    value = value.to_dict()['Lp']
    year_sales_dict_krk[key] = value
    
for key, value in year_sales_dict_week_krk.items():
    value = value.to_dict()['Lp']
    year_sales_dict_week_krk[key] = value
    
for key, value in year_sales_dict_wr.items():
    value = value.to_dict()['Lp']
    year_sales_dict_wr[key] = value
    
for key, value in year_sales_dict_week_wr.items():
    value = value.to_dict()['Lp']
    year_sales_dict_week_wr[key] = value
    
#     odszumienie zbioru danych dla calego kraju
            
for index, row in date_sales_un.iterrows():
    year = row['ds'].date().year
    month = row['ds'].date().month
    week = row['ds'].date().isocalendar()[1]
    
    avg_month_sales = year_sales_dict[year][month]/30
    
    if week in year_sales_dict_week[year].keys():
        avg_week_sales = year_sales_dict_week[year][week]/6
    
    else:
        avg_week_sales = avg_month_sales
    
#     interpolacja wartosciami okolicznymi jesli wartosc mniejsza niz 15% sredniej
    
    if row['y'] <= 0.15*avg_month_sales:
        date_sales_un.at[index, 'y'] = int(avg_week_sales)
        
#     odszumienie zbioru danych dla warszawy
            
for index, row in date_sales_wwa_un.iterrows():
    year = row['ds'].date().year
    month = row['ds'].date().month
    week = row['ds'].date().isocalendar()[1]
    
    avg_month_sales = year_sales_dict_wwa[year][month]/30
    
    if week in year_sales_dict_week_wwa[year].keys():
        avg_week_sales = year_sales_dict_week_wwa[year][week]/6
    
    else:
        avg_week_sales = avg_month_sales
    
#     interpolacja wartosciami okolicznymi jesli wartosc mniejsza niz 15% sredniej
    
    if row['y'] <= 0.15*avg_month_sales:
        date_sales_wwa_un.at[index, 'y'] = int(avg_week_sales)

        
#     odszumienie zbioru danych dla calego kraju
            
for index, row in date_sales_krk_un.iterrows():
    year = row['ds'].date().year
    month = row['ds'].date().month
    week = row['ds'].date().isocalendar()[1]
    
    avg_month_sales = year_sales_dict_krk[year][month]/30
    
    if week in year_sales_dict_week_krk[year].keys():
        avg_week_sales = year_sales_dict_week_krk[year][week]/6
    
    else:
        avg_week_sales = avg_month_sales
    
#     interpolacja wartosciami okolicznymi jesli wartosc mniejsza niz 15% sredniej
    
    if row['y'] <= 0.15*avg_month_sales:
        date_sales_krk_un.at[index, 'y'] = int(avg_week_sales)

        
#     odszumienie zbioru danych dla calego kraju
            
for index, row in date_sales_wr_un.iterrows():
    year = row['ds'].date().year
    month = row['ds'].date().month
    week = row['ds'].date().isocalendar()[1]
    
    avg_month_sales = year_sales_dict_wr[year][month]/30
    
    if week in year_sales_dict_week_wr[year].keys():
        avg_week_sales = year_sales_dict_week_wr[year][week]/6
    
    else:
        avg_week_sales = avg_month_sales
    
#     interpolacja wartosciami okolicznymi jesli wartosc mniejsza niz 15% sredniej
    
    if row['y'] <= 0.15*avg_month_sales:
        date_sales_wr_un.at[index, 'y'] = int(avg_week_sales)


WARNING - (py.warnings._showwarnmsg) - /tmp/ipykernel_94068/1141912089.py:55: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  year_sales_dict_week[year]=df[df['DataRozpoczecia'].dt.year == year].groupby(df['DataRozpoczecia'].dt.week).agg({"Lp":"sum"})

WARNING - (py.warnings._showwarnmsg) - /tmp/ipykernel_94068/1141912089.py:58: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  year_sales_dict_week_wwa[year]=df_warszawa[df_warszawa['DataRozpoczecia'].dt.year == year].groupby(df_warszawa['DataRozpoczecia'].dt.week).agg({"Lp":"sum"})

WARNING - (py.warnings._showwarnmsg) - /tmp/ipykernel_94068/1141912089.py:61: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  year_sales_dict_week_krk[year]=df_krakow[df_krakow['DataRozpoczecia'].dt.year == year].groupby(df

In [78]:
# detekcja anomalii
weekdays_data=df.groupby(df['day_of_week']).agg({"Lp":"sum"})
weekdays_data_avg=df.groupby(df['day_of_week']).agg({"Lp":"sum"})
weekday_occ_dict = {}
first_date = df.head(1)['DataRozpoczecia'].dt.date.values[0]
last_date = df.tail(1)['DataRozpoczecia'].dt.date.values[0]
dates = df['DataRozpoczecia'].dt.date.unique()
for date in dates:
    if date.weekday()+1 not in weekday_occ_dict.keys():
        weekday_occ_dict[date.weekday()+1] = 1
    else:
        weekday_occ_dict[date.weekday()+1] += 1
# print(dates)

print(weekday_occ_dict)

for index, row in weekdays_data.iterrows():
    weekdays_data_avg.at[index, 'Lp'] = row['Lp']/weekday_occ_dict[index+1]
    

{4: 90, 6: 90, 2: 94, 1: 90, 5: 91, 3: 93, 7: 21}


,Lp
day_of_week,
0,289.488889
1,271.723404
2,254.397849
3,252.966667
4,320.461538
